In [1]:
import pandas as pd
from shapely.geometry import Point
from pyproj import Transformer
import geopandas as gpd

# Read data of property
property_data = pd.read_csv('../data/raw/location_rent_with_coordinates.csv')

# Read data of stations (tram, train, bus)
tram_stops = gpd.read_file('../data/raw/Transportation/PTV_METRO_TRAM_STOP.shp')
train_stops = gpd.read_file('../data/raw/Transportation/PTV_METRO_TRAIN_STATION.shp')
bus_stops = gpd.read_file('../data/raw/Transportation/PTV_METRO_BUS_STOP.shp')

# Read data of school and encoding to 'ISO-8859-1'
school_data = pd.read_csv('../data/raw/schoollocations2023.csv', encoding='ISO-8859-1')

# Initialize the transformer to convert WGS84 (longitude and latitude) to EPSG:3111 (Vicgrid)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3111", always_xy=True)

# Convert property longitude and latitude to EPSG:3111
property_data['geometry'] = property_data.apply(
    lambda row: Point(transformer.transform(row['longitude'], row['latitude'])), axis=1
)

# Convert property data to a GeoDataFrame
property_gdf = gpd.GeoDataFrame(property_data, geometry='geometry', crs='EPSG:3111')

# Ensure valid geometry for tram, train, and bus stops
tram_stops['geometry'] = tram_stops['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))
train_stops['geometry'] = train_stops['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))
bus_stops['geometry'] = bus_stops['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

# Convert school longitude and latitude to EPSG:3111 and create a GeoDataFrame
school_data['geometry'] = school_data.apply(
    lambda row: Point(transformer.transform(row['X'], row['Y'])), axis=1
)
school_gdf = gpd.GeoDataFrame(school_data, geometry='geometry', crs='EPSG:3111')

# Calculate the nearest tram stop and its distance for each property
def get_nearest_tram_stop(geom):
    if geom is None or geom.is_empty:
        return None, None
    tram_distances = tram_stops.distance(geom)
    nearest_tram_idx = tram_distances.idxmin()
    nearest_tram_stop_name = tram_stops.loc[nearest_tram_idx, 'STOP_NAME']
    nearest_tram_distance_km = tram_distances.min() / 1000 
    return nearest_tram_stop_name, nearest_tram_distance_km

# Apply the nearest tram stop and distance calculation
property_gdf['nearest_tram_stop'], property_gdf['nearest_tram_stop_distance_km'] = zip(
    *property_gdf.geometry.apply(lambda geom: get_nearest_tram_stop(geom) if geom.is_valid else (None, None))
)

# Calculate the nearest train stop and its distance for each property
def get_nearest_train_stop(geom):
    if geom is None or geom.is_empty:
        return None, None
    train_distances = train_stops.distance(geom)
    nearest_train_idx = train_distances.idxmin()
    nearest_train_stop_name = tram_stops.loc[nearest_train_idx, 'STOP_NAME']
    nearest_train_distance_km = train_distances.min() / 1000  
    return nearest_train_stop_name, nearest_train_distance_km

# Apply the nearest train stop and distance calculation
property_gdf['nearest_train_stop'], property_gdf['nearest_train_stop_distance_km'] = zip(
    *property_gdf.geometry.apply(lambda geom: get_nearest_train_stop(geom) if geom.is_valid else (None, None))
)

# Calculate the nearest bus stop and its distance for each property
def get_nearest_bus_stop(geom):
    if geom is None or geom.is_empty:
        return None, None
    bus_distances = bus_stops.distance(geom)
    nearest_bus_idx = bus_distances.idxmin()
    nearest_bus_stop_name = bus_stops.loc[nearest_bus_idx, 'STOP_NAME']
    nearest_bus_distance_km = bus_distances.min() / 1000 
    return nearest_bus_stop_name, nearest_bus_distance_km

# Apply the nearest bus stop and distance calculation
property_gdf['nearest_bus_stop'], property_gdf['nearest_bus_stop_distance_km'] = zip(
    *property_gdf.geometry.apply(lambda geom: get_nearest_bus_stop(geom) if geom.is_valid else (None, None))
)

# Calculate the nearest school and its distance for each property
def get_nearest_school(geom):
    if geom is None or geom.is_empty:
        return None, None
    distances = school_gdf.distance(geom)
    nearest_idx = distances.idxmin()
    nearest_school_name = school_gdf.loc[nearest_idx, 'School_Name']  
    nearest_school_distance_km = distances.min() / 1000  
    return nearest_school_name, nearest_school_distance_km

# Apply the nearest school and distance calculation
property_gdf['nearest_school'], property_gdf['nearest_school_distance_km'] = zip(
    *property_gdf.geometry.apply(lambda geom: get_nearest_school(geom) if geom.is_valid else (None, None))
)

# Check
property_gdf.head(10)

,name,Bed,weekly_rent,latitude,longitude,geometry,nearest_tram_stop,nearest_tram_stop_distance_km,nearest_train_stop,nearest_train_stop_distance_km,nearest_bus_stop,nearest_bus_stop_distance_km,nearest_school,nearest_school_distance_km
0,"1208/50 Albert Street, South Melbourne VIC 3205",1,520.0,-37.834344,144.955904,POINT (2496118.379 2407409.798),127-South Melbourne Station/Light Rail (South ...,0.136816,67-Camberwell Girls Grammar/Burke Rd (Hawthorn...,1.863573,Ferrars St/Dorcas St (South Melbourne),0.034790,Galilee Regional Catholic Primary School,0.223329
1,"64 Mills Street, Albert Park VIC 3206",3,1495.0,-37.846426,144.958009,POINT (2496304.265 2406068.919),135-Richardson St/Mills St (Middle Park),0.090635,49-Buchanan Ave/Doncaster Rd (Balwyn North),2.824165,Middle Park PS/Richardson St (Middle Park),0.095911,Middle Park Primary School,0.158652
2,"11 Barnato St, Weir Views VIC 3338",4,460.0,-37.718775,144.554187,POINT (2460697.114 2420145.173),49-Central Park Ave/Cordite Ave (Maribyrnong),29.072242,46-Orrong Rd/Glenhuntly Rd (Elsternwick),17.817118,Hume Ave/Rees Rd (Melton South),1.047037,Al Iman College,1.632357
3,"1104/70 Southbank Boulevard, Southbank VIC 3006",1,420.0,-37.823158,144.963953,POINT (2496826.489 2408651.663),115-Casino/Southbank/Queens Bridge St (Southbank),0.266535,41-Cantala Ave/Dandenong Rd (Armadale),0.600295,City Rd/Southbank Bvd (Southbank),0.061561,Victorian College Of The Arts Secondary School,0.567526
4,"167 Charman Road, Beaumaris VIC 3193",4,950.0,-37.974606,145.053631,POINT (2504712.148 2391841.452),68-East Brighton/Hawthorn Rd (Brighton East),7.439762,68-Mont Albert Rd/Burke Rd (Camberwell),0.886863,Keith St/Charman Rd (Beaumaris),0.089153,Mentone Girls Secondary College,0.618751
5,"10/135 Ormond Esplanade, Elwood VIC 3184",3,1200.0,-37.890515,144.989992,POINT (2499119.704 2401176.235),43-Brighton Rd/Glenhuntly Rd (Elsternwick),0.991842,43-Princes Tce/Melville Rd (Pascoe Vale South),1.152975,Ormond Esp/St Kilda St (Elwood),0.076032,Elsternwick Primary School,0.649723
6,"2/107 Addison Street, Elwood VIC 3184",3,895.0,-37.880749,144.980150,POINT (2498253.739 2402260.045),39-Glen Eira Rd/Brighton Rd (Elwood),1.171650,44-Brearley Pde/Melville Rd (Pascoe Vale South),1.432020,Addison St/Glen Huntly Rd (Elwood),0.128267,St Columba's School,0.215275
7,"3/18 Joyce Street, Elwood VIC 3184",2,870.0,-37.889879,144.990439,POINT (2499158.98 2401246.918),43-Brighton Rd/Glenhuntly Rd (Elsternwick),0.913309,43-Princes Tce/Melville Rd (Pascoe Vale South),1.081718,Ormond Esp/St Kilda St (Elwood),0.094843,Elsternwick Primary School,0.626450
8,"2/435 St Kilda St, Elwood VIC 3184",2,675.0,-37.890244,144.990787,POINT (2499189.656 2401206.354),43-Brighton Rd/Glenhuntly Rd (Elsternwick),0.923036,43-Princes Tce/Melville Rd (Pascoe Vale South),1.078153,Ormond Esp/St Kilda St (Elwood),0.047875,Elsternwick Primary School,0.586666
9,"305/101 Bay Street, Port Melbourne VIC 3207",1,595.0,-37.840640,144.940024,POINT (2494721 2406710.307),128-Graham St/Light Rail (Port Melbourne),0.459501,67-Camberwell Girls Grammar/Burke Rd (Hawthorn...,2.711914,Nott St/Graham St (Port Melbourne),0.097550,Port Phillip Specialist School,0.460344


In [2]:
#Save to csv
property_gdf.to_csv('property_with_nearest_distance.csv', index=False)